# Simulator Integration

This tutorial shows how to integrate a simulator into OpenSBT, in particular how to write an adapter for a simulation. As an example we use the implemented adapter CarlaRunner for the CARLA Simulator.

((Show the strucutre of simout))

((Sim adapter shall convert output in simout))

((Inteface sim adapter OpenSBT))

((Extension of simout))